In [8]:
# spikingjelly.activation_based.examples.conv_fashion_mnist
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from spikingjelly.activation_based import encoding, monitor
from spikingjelly.activation_based import neuron, functional, surrogate, layer
from torch.utils.tensorboard import SummaryWriter
from spikingjelly.activation_based import lava_exchange
#Even trying to import gives an error?
# from spikingjelly.activation_based.lava_exchange import to_lava_neuron, linear_to_lava_synapse_dense
import os
import time
import argparse
from torch.cuda import amp
import sys
import datetime
from spikingjelly import visualizing
import numpy as np

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

ImportError: cannot import name 'to_lava_neuron' from 'spikingjelly.activation_based.lava_exchange' (c:\Users\Nathan\.conda\envs\python_env1\lib\site-packages\spikingjelly\activation_based\lava_exchange.py)

Model classes to load up

In [2]:
class SCNN(nn.Module):
    def __init__(self, T: int):
        super().__init__()
        self.T = T
        self.conv_and_fc = nn.Sequential(
            layer.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2, bias=False),
            layer.BatchNorm2d(6),
            neuron.LIFNode(surrogate_function=surrogate.ATan()),
            layer.MaxPool2d(kernel_size=2, stride=2),

            layer.Conv2d(in_channels=6, out_channels=12, kernel_size=5, bias=False),
            layer.BatchNorm2d(12),
            neuron.LIFNode(surrogate_function=surrogate.ATan()),
            layer.MaxPool2d(kernel_size=2, stride=2),

            layer.Flatten(),
            layer.Linear(5*5*12, 10, bias=False),
            neuron.LIFNode(surrogate_function=surrogate.ATan()),
            # layer.Linear(120, 84),
            # neuron.IFNode(surrogate_function=surrogate.LeakyKReLU()),
            # layer.Linear(84, 10),
            # neuron.IFNode(surrogate_function=surrogate.ATan())
            )
        functional.set_step_mode(self, step_mode='m')
    def forward(self, x):
        x_seq = x.unsqueeze(0).repeat(self.T, 1, 1, 1, 1)
        z = self.conv_and_fc(x_seq)
        fr = z.mean(0)
        return fr

In [3]:
class SNN(nn.Module):
    def __init__(self, tau):
        super().__init__()
        self.layer1 = nn.Sequential(
            layer.Linear(28*28, 10, bias=False),
            neuron.LIFNode(tau=tau, surrogate_function=surrogate.ATan())
        )
    def forward(self, x: torch.Tensor):
        # print(x.shape)
        return self.layer1(x)

In [4]:
single_model = SNN(tau=2.0)
file_dir = './Models/'
full_path = file_dir + '/SingleLayer_SNN.pt'
checkpoint = torch.load(f=full_path)
single_model.load_state_dict(checkpoint)

<All keys matched successfully>

In [1]:
#I do not understand why none of the functions are accessible
#Functions are in the file, they just can't be called
single_lava = single_model
print(lava_exchange.linear_to_lava_synapse_dense())
single_lava.conv_and_fc[0] = lava_exchange.linear_to_lava_synapse_dense(single_model.layer1[0])
single_lava.conv_and_fc[1] = lava_exchange.to_lava_neuron(single_model.layer1[1])

NameError: name 'single_model' is not defined